# Setup & Utils

In [ ]:
import requests

In [ ]:
import re

def extract_model_response(text: str) -> str:
    """
    Extracts the content between <think> and the final <|eot_id|> marker.

    Parameters:
        text (str): The full text containing <think> and <|eot_id|> markers.

    Returns:
        str: The extracted content between <think> and <|eot_id|>, or an empty string if not found.
    """
    match = re.search(r"<think>(.*?)<\|end_of_text\|>", text, re.DOTALL)
    return match.group(1).strip() if match else ""


**Please enter the API key that was provided to you**

In [ ]:
API_KEY = ""

# Use Case 1 - MITRE Mapping Detection Plays

**Use Case Description** - Security teams often write detection rules or response playbooks (e.g., for SIEMs, SOAR platforms, or XDRs) but struggle to systematically map them to the right MITRE ATT&CK techniques. This limits the organization’s visibility into detection coverage and hinders frameworks like D3FEND, ATT&CK Navigator, and threat-informed defense.

The goal is to automatically analyze each detection play’s logic, objective, and context — and match it to the most relevant ATT&CK technique(s) with high confidence.

## MITRE Mapping Utils

In [ ]:
def create_mitre_mapping_prompt(app_name, detection_plays):
    """Generate a prompt for mapping detection plays to MITRE techniques."""
    return f"""
You are a cybersecurity analyst provided with a series of threat detection plays. Each play includes its name and a short objective.

Your task: For each detection play, map it to the most relevant MITRE ATT&CK technique. Include:
1. MITRE technique ID and name
2. Justification for the mapping

## DETECTION PLAYS
{detection_plays}

Respond in a structured format:
- Play Name: <name>
- MITRE Technique: <TXXXX: Technique Name>
- Reason: <why it's a fit>
"""

In [ ]:
def run_mitre_mapping_analysis(app_name, detection_plays):
    """Send detection plays to AI model and return MITRE mappings."""
    prompt = create_mitre_mapping_prompt(app_name, detection_plays)

    resp = requests.post(
        "https://model-dq4r501w.api.baseten.co/development/predict",
        headers={"Authorization": f"Api-Key {API_KEY}"},
        json={"prompt": prompt},
    )

    return extract_model_response(resp.text)

## Example Usage

In [ ]:
sample_detection_plays = """
- Name: Monitor Multiple Failed Login Attempts
  Objective: Detect brute-force attacks on user accounts by tracking repeated login failures.

- Name: Alert on PowerShell Script Execution
  Objective: Detect execution of potentially malicious PowerShell scripts by adversaries.

- Name: Dumping LSASS credentials
  Objective: Alert when attempts are made to dump credentials from LSASS process memory
"""


In [ ]:
output = run_mitre_mapping_analysis("Finance Reporting App", sample_detection_plays)


In [ ]:
print(output)


---

**Play Name:** Monitor Multiple Failed Login Attempts  
**MITRE Technique:** T1110.001 - Brute Force: Password Guessing  
**Reason:** This play is designed to detect brute-force attacks on user accounts by tracking repeated login failures. The MITRE ATT&CK technique T1110.001 specifically refers to the use of password guessing (brute force) where an attacker repeatedly tries different passwords until they find the correct one. Monitoring multiple failed login attempts aligns directly with this technique as it is a common method used by attackers to gain unauthorized access through trial-and-error authentication attempts.

---

**Play Name:** Alert on PowerShell Script Execution  
**MITRE Technique:** T1059.003 - Command and Scripting Interpreter: PowerShell  
**Reason:** The objective here is to detect execution of potentially malicious PowerShell scripts. PowerShell is a powerful scripting environment that can be abused by adversaries to execute commands, scripts, or payloads. Th